# **Bussiness dan Data Understanding**

Tujuan project kali ini
1. Menentukan spesifikasi mobil yang akan saya rekomendasikan kepada user
2. Menentukan harga yangs sesuai dengan spesifikasi mobil yang akan direkomendasikan kepada user supaya laku di pasaran



## Library yang akan kita pakai

In [84]:
import pandas as pd
import numpy as np
import plotly.express as px


## Load data

In [85]:
df = pd.read_csv('/content/Car_sales.csv')
df.head()

,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
0,Acura,Integra,16.919,16.360,Passenger,21.50,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2/2/2012,58.280150
1,Acura,TL,39.384,19.875,Passenger,28.40,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,6/3/2011,91.370778
2,Acura,CL,14.114,18.225,Passenger,NaN,3.2,225.0,106.9,70.6,192.0,3.470,17.2,26.0,1/4/2012,NaN
3,Acura,RL,8.588,29.725,Passenger,42.00,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,3/10/2011,91.389779
4,Audi,A4,20.397,22.255,Passenger,23.99,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,10/8/2011,62.777639


## Melihat info data

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Manufacturer         157 non-null    object 
 1   Model                157 non-null    object 
 2   Sales_in_thousands   157 non-null    float64
 3   __year_resale_value  121 non-null    float64
 4   Vehicle_type         157 non-null    object 
 5   Price_in_thousands   155 non-null    float64
 6   Engine_size          156 non-null    float64
 7   Horsepower           156 non-null    float64
 8   Wheelbase            156 non-null    float64
 9   Width                156 non-null    float64
 10  Length               156 non-null    float64
 11  Curb_weight          155 non-null    float64
 12  Fuel_capacity        156 non-null    float64
 13  Fuel_efficiency      154 non-null    float64
 14  Latest_Launch        157 non-null    object 
 15  Power_perf_factor    155 non-null    flo

# **Data Preparation**

## Menangani null values

jika kita melihat dari info dari data, terdapat beberapa values yang nilainya null. kita akan menangani setiap kolomnya satu persatu.
Untuk pertama kita akan melihat data yang ada di kolom Engine_Size nya yang memiliki nilai null. Kita akan melakukannya dengan langkah berikut:

In [87]:
df[df['Engine_size'].isna() ==True]

,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
33,Chrysler,Town & Country,53.48,19.54,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/13/2011,NaN


Setelah dilihat, ternyata data ini memiliki beberapa kolom penting yang valuenya null. akan lebih baik kita menghapus seluruh baris tersebut supaya tidak mengganngu proses analisa dan pembuatan model. kita akan melakukan drop data pada baris tersebut

In [88]:
df = df.drop(df[df['Engine_size'].isna()==True].index)
df.isna().sum()

,0
Manufacturer,0
Model,0
Sales_in_thousands,0
__year_resale_value,36
Vehicle_type,0
Price_in_thousands,1
Engine_size,0
Horsepower,0
Wheelbase,0
Width,0


Dapat kita lihat value null pada Engine_size sudah tidak ada

kemudian pada Price_in_thousands kita handling juga, karena harga akan menjadi target dari model prediksi.

In [89]:
df[df['Price_in_thousands'].isna() == True]

,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
2,Acura,CL,14.114,18.225,Passenger,NaN,3.2,225.0,106.9,70.6,192.0,3.47,17.2,26.0,1/4/2012,NaN


Terdapat satu baris dan untuk harga kita hanya akan mengisinya dengan angka rata" dari keseluruhan harga mobil

In [90]:
df['Price_in_thousands'] = df['Price_in_thousands'].fillna(df['Price_in_thousands'].mean())
df.isna().sum()

,0
Manufacturer,0
Model,0
Sales_in_thousands,0
__year_resale_value,36
Vehicle_type,0
Price_in_thousands,0
Engine_size,0
Horsepower,0
Wheelbase,0
Width,0


Null values pada Price in Thousands sudah di tangani

Hanya tinggal satu kolom yaitu __year_resale_value
kolom tersebut menunjukan harga resale atau harga mobil ketika di jual dalam kondisi bekas. kita akan memanfaatkan data harga mobil atau price in thousands. dengan mengurangi price_in_thousands dengan _year_resale_value supaya kita bisa lihat perbedaan harga antara mobil kondisi baru dengan kondisi bekas. perbedaan itu kemudian akan kita rata-ratakan. nilai tersebut yang akan kita gunakan untuk menentukan harga bekasa dari masing-masing mobil yang kolom _year_resale_valuenya null.

In [91]:
# Isi NaN dengan mean masing-masing kolom
df['Curb_weight'] = df['Curb_weight'].fillna(df['Curb_weight'].mean())
df['Fuel_efficiency'] = df['Fuel_efficiency'].fillna(df['Fuel_efficiency'].mean())
df['Power_perf_factor'] = df['Power_perf_factor'].fillna(df['Power_perf_factor'].mean())

# Cek sisa missing value
df.isna().sum()


,0
Manufacturer,0
Model,0
Sales_in_thousands,0
__year_resale_value,36
Vehicle_type,0
Price_in_thousands,0
Engine_size,0
Horsepower,0
Wheelbase,0
Width,0


In [92]:
# melihat datanya
df[df['__year_resale_value'].isna() == True]

,Manufacturer,Model,Sales_in_thousands,__year_resale_value,Vehicle_type,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Latest_Launch,Power_perf_factor
7,BMW,323i,19.747,NaN,Passenger,26.990,2.5,170.0,107.3,68.4,176.0,3.179,16.6,26.000000,6/28/2011,71.191207
18,Cadillac,Escalade,14.785,NaN,Car,46.225,5.7,255.0,117.5,77.0,201.2,5.572,30.0,15.000000,4/17/2012,109.509117
27,Chevrolet,Impala,107.995,NaN,Passenger,18.890,3.4,180.0,110.5,73.0,200.0,3.389,17.0,27.000000,6/18/2011,71.838039
34,Chrysler,300M,30.696,NaN,Passenger,29.185,3.5,253.0,113.0,74.4,197.8,3.567,17.0,23.000000,2/10/2012,101.655244
44,Dodge,Durango,101.323,NaN,Car,26.310,5.2,230.0,115.7,71.7,193.5,4.394,25.0,17.000000,6/27/2012,92.854125
50,Ford,Focus,175.670,NaN,Passenger,12.315,2.0,107.0,103.0,66.9,174.8,2.564,13.2,30.000000,7/22/2012,43.117132
66,Jaguar,S-Type,15.467,NaN,Passenger,42.800,3.0,240.0,114.5,71.6,191.3,3.650,18.4,21.000000,11/3/2012,102.178985
72,Lexus,GS400,3.334,NaN,Passenger,46.305,4.0,300.0,110.2,70.9,189.2,3.693,19.8,21.000000,11/28/2012,125.013357
74,Lexus,LX470,9.126,NaN,Car,60.105,4.7,230.0,112.2,76.4,192.5,5.401,25.4,15.000000,10/30/2012,105.760458
75,Lexus,RX300,51.238,NaN,Car,34.605,3.0,220.0,103.0,71.5,180.1,3.900,17.2,21.000000,1/4/2012,91.943802


In [93]:
#perbedaan harga baru dengan harga bekas
df[['Price_in_thousands','__year_resale_value']]

,Price_in_thousands,__year_resale_value
0,21.500000,16.360
1,28.400000,19.875
2,27.390755,18.225
3,42.000000,29.725
4,23.990000,22.255
...,...,...
152,24.400000,NaN
153,27.500000,NaN
154,28.800000,NaN
155,45.500000,NaN


In [94]:
#membuat variabel untuk menyimpan nilai rata-rata dari perbedaan harga mobil baru dan bekas
reduction_avg = pd.Series(df['Price_in_thousands'] - df['__year_resale_value']).mean()
reduction_avg

np.float64(8.045922956989248)

nilai rata-rata perbedaan harga mobil baru dan bekas adalah 8.045922956989248. kemudian nilai ini akan kita gunakan untuk mengurangi harga mobil baru pada data dengan kolom __year_resale_value yang null

In [95]:
df['__year_resale_value'] = df['__year_resale_value'].fillna(df['Price_in_thousands'] - reduction_avg)
df.isna().sum()

,0
Manufacturer,0
Model,0
Sales_in_thousands,0
__year_resale_value,0
Vehicle_type,0
Price_in_thousands,0
Engine_size,0
Horsepower,0
Wheelbase,0
Width,0


# Story Telling

## Model dengan penjualan terbaik

In [96]:
df_model = df[['Model','Manufacturer', 'Sales_in_thousands']].groupby(['Model','Manufacturer']).sum().reset_index()
df_model = df_model.sort_values(by='Sales_in_thousands', ascending=False)
df_model.head(10)

,Model,Manufacturer,Sales_in_thousands
63,F-Series,Ford,540.561
62,Explorer,Ford,276.747
31,Camry,Toyota,247.994
146,Taurus,Ford,245.815
12,Accord,Honda,230.902
118,Ram Pickup,Dodge,227.061
121,Ranger,Ford,220.650
41,Civic,Honda,199.685
32,Caravan,Dodge,181.749
65,Focus,Ford,175.670


In [97]:
fig = px.bar(df_model.head(10), x='Model', y='Sales_in_thousands', color='Model', text_auto= True, title = 'Top 10 Model dengan penjualan terbaik')
fig.update_traces(textfont_size=10, textposition="inside")
fig.update_layout(width = 1000, height = 800)
fig.show()

Dapat kita lihat, di atas merupakan sepuluh model mobil yang memiliki penjualan terbaik dipasaran. F-series memimpin dengan lebih dari 500 ribu penjualan, disusul oleh model explorer dan carmy. dilihat dari manufacture, ford mendominasi daftar sepuluh penjualan terbaik dengan 5 model mobilnya. nantinya 10 model ini dapat menjadi acuan kita, mobil seperti apa yang terbukti laris di pasaran.

## Membuat data frame untuk melakukan analisa fitur-fitur yang dimiliki model

In [98]:
col_selected = ['Model','Vehicle_type', '__year_resale_value', 'Price_in_thousands', 'Engine_size', 'Horsepower', 'Wheelbase', 'Width', 'Length', 'Curb_weight', 'Fuel_capacity', 'Fuel_efficiency', 'Power_perf_factor']

In [99]:
df_selected = df[col_selected].groupby(['Model', 'Vehicle_type']).mean( )
df_selected.head()

,,__year_resale_value,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Power_perf_factor
Model,Vehicle_type,,,,,,,,,,,
3-Sep,Passenger,18.054077,26.100,2.0,185.0,102.6,67.4,182.2,2.990,16.9,23.0,76.023048
3000GT,Passenger,20.940000,25.450,3.0,161.0,97.2,72.4,180.3,3.131,19.8,21.0,67.544155
300M,Passenger,21.139077,29.185,3.5,253.0,113.0,74.4,197.8,3.567,17.0,23.0,101.655244
323i,Passenger,18.944077,26.990,2.5,170.0,107.3,68.4,176.0,3.179,16.6,26.0,71.191207
328i,Passenger,28.675000,33.400,2.8,193.0,107.3,68.5,176.0,3.197,16.6,24.0,81.877069


In [100]:
# merge data dengan df model
df_to_analyze = df_selected.merge(df_model, on='Model')
df_to_analyze.sort_values(by='Sales_in_thousands', ascending=False, inplace=True)


## Harga mobil dengan penjualan terbaik

In [101]:
df_to_analyze.head(10)

,Model,__year_resale_value,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Power_perf_factor,Manufacturer,Sales_in_thousands
63,F-Series,15.075000,26.935,4.6,220.0,138.5,79.1,224.5,4.241,25.1,18.0,89.401935,Ford,540.561
62,Explorer,16.640000,31.930,4.0,210.0,111.6,70.2,190.7,3.876,21.0,19.0,87.635496,Ford,276.747
31,Camry,13.245000,17.518,2.2,133.0,105.2,70.1,188.5,2.998,18.5,27.0,54.372420,Toyota,247.994
146,Taurus,10.055000,17.885,3.0,155.0,108.5,73.0,197.6,3.368,16.0,24.0,62.503740,Ford,245.815
12,Accord,13.210000,15.350,2.3,135.0,106.9,70.3,188.8,2.932,17.1,27.0,54.269548,Honda,230.902
118,Ram Pickup,15.060000,19.460,5.2,230.0,138.7,79.3,224.2,4.470,26.0,17.0,90.211700,Dodge,227.061
121,Ranger,7.850000,12.050,2.5,119.0,117.5,69.4,200.7,3.086,20.0,23.0,47.389531,Ford,220.650
41,Civic,9.850000,12.885,1.6,106.0,103.2,67.1,175.1,2.339,11.9,32.0,42.879097,Honda,199.685
32,Caravan,12.025000,19.565,2.4,150.0,113.3,76.8,186.3,3.533,20.0,24.0,61.227000,Dodge,181.749
65,Focus,4.269077,12.315,2.0,107.0,103.0,66.9,174.8,2.564,13.2,30.0,43.117132,Ford,175.670


In [102]:
fig = px.bar(df_to_analyze.head(10), x='Price_in_thousands', y='Model', color='Model',orientation = 'h', text_auto= True, title = 'Harga dari 10 model terbaik')
fig.update_traces(textfont_size=10, textposition="outside")
fig.update_layout(width = 1000, height = 800)
fig.show()

## Angka Horsepower yang dimiliki oleh model dengan penjualan terbaik

In [103]:
fig = px.bar(df_to_analyze.head(10), x='Horsepower', y='Model', color='Model',orientation = 'h', text_auto= True, title = 'Angka Horsepower dari 10 model penjualan terbaik')
fig.update_traces(textfont_size=10, textposition="outside")
fig.update_layout(width = 1000, height = 800)
fig.show()

# Menentukan spesifikasi mobil yang hendak diproduksi

In [104]:
df_to_analyze.head(10)

,Model,__year_resale_value,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Power_perf_factor,Manufacturer,Sales_in_thousands
63,F-Series,15.075000,26.935,4.6,220.0,138.5,79.1,224.5,4.241,25.1,18.0,89.401935,Ford,540.561
62,Explorer,16.640000,31.930,4.0,210.0,111.6,70.2,190.7,3.876,21.0,19.0,87.635496,Ford,276.747
31,Camry,13.245000,17.518,2.2,133.0,105.2,70.1,188.5,2.998,18.5,27.0,54.372420,Toyota,247.994
146,Taurus,10.055000,17.885,3.0,155.0,108.5,73.0,197.6,3.368,16.0,24.0,62.503740,Ford,245.815
12,Accord,13.210000,15.350,2.3,135.0,106.9,70.3,188.8,2.932,17.1,27.0,54.269548,Honda,230.902
118,Ram Pickup,15.060000,19.460,5.2,230.0,138.7,79.3,224.2,4.470,26.0,17.0,90.211700,Dodge,227.061
121,Ranger,7.850000,12.050,2.5,119.0,117.5,69.4,200.7,3.086,20.0,23.0,47.389531,Ford,220.650
41,Civic,9.850000,12.885,1.6,106.0,103.2,67.1,175.1,2.339,11.9,32.0,42.879097,Honda,199.685
32,Caravan,12.025000,19.565,2.4,150.0,113.3,76.8,186.3,3.533,20.0,24.0,61.227000,Dodge,181.749
65,Focus,4.269077,12.315,2.0,107.0,103.0,66.9,174.8,2.564,13.2,30.0,43.117132,Ford,175.670


In [105]:
df_prediction = df_to_analyze.drop (['Model', '__year_resale_value', 'Price_in_thousands', 'Manufacturer', 'Sales_in_thousands'],axis = 1)
df_prediction.mean().round()

,0
Engine_size,3.0
Horsepower,186.0
Wheelbase,107.0
Width,71.0
Length,187.0
Curb_weight,3.0
Fuel_capacity,18.0
Fuel_efficiency,24.0
Power_perf_factor,77.0


data diatas adalah rata-rata spesifikasi model mobil dan bisa digunakan untuk acuan produksi mobil

# Model prediksi harga mobil

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

## Feature Selection

Digunakan untuk menyeleksi kolom apa saja yang akan dipakai untuk prediksi

Variable Prediktor / independent


*   Engine_size
*   Horsepower
* Whellbase
* Width
* Length
* Curb_Weight
* Fuel_capacity
* Fuel_Efficiency
* Power_perf_factor

Variable Dependent/Target

* Price_in_thousands



In [107]:
pred_data = df_to_analyze.drop(['Model', '__year_resale_value', 'Manufacturer', 'Sales_in_thousands'], axis=1)
pred_data.head()

,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Power_perf_factor
63,26.935,4.6,220.0,138.5,79.1,224.5,4.241,25.1,18.0,89.401935
62,31.930,4.0,210.0,111.6,70.2,190.7,3.876,21.0,19.0,87.635496
31,17.518,2.2,133.0,105.2,70.1,188.5,2.998,18.5,27.0,54.372420
146,17.885,3.0,155.0,108.5,73.0,197.6,3.368,16.0,24.0,62.503740
12,15.350,2.3,135.0,106.9,70.3,188.8,2.932,17.1,27.0,54.269548


## Memisahkan Variable Independent dan Dependent

X = Variable Independent
y = Variable Dependent

In [108]:
x = pred_data.drop('Price_in_thousands', axis=1)
y = pred_data['Price_in_thousands']

In [109]:
x.shape

(156, 9)

## Train test split

In [110]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Membuat model Prediksi

Kita akan pakai Multiple Linier Regresion dimana bisa menghubungkan satu var dependent dengan dua atau lebih var independent. tujuannya adalah prediksi nilai variable dependent berdasar nilai variable independent yang ada.

In [111]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

## Melakukan Evaluasi Model

In [113]:
y_pred = model.predict(x_test)
y_pred

array([22.51570812, 21.50763923, 25.68429089, 31.9670488 , 37.4895899 ,
       10.40542596, 11.61339168, 40.15858797, 47.01214156, 16.55553592,
       20.19308323, 18.96082481, 19.8607866 , 50.09218599, 23.96486346,
       22.36244924, 29.02608831, 33.03537774, 17.23293844, 37.46063521,
       26.39221772, 12.72358899, 12.08795701, 11.92785315, 18.33207148,
       28.67175964, 17.95854315, 24.29503006, 22.04825048, 31.13538758,
       21.35889437, 40.27446622])

Selanjutnya kita akan mengukur sejauh mana model regresi akurat atau tidak, kita akan pakai

Root Mean Squared Error (RMSE) : Semakin rendah nilainya makan semakin baik model kita


R2 Score : semakin tinggi nilai maka semakin baik model kita

In [121]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2score = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R²:", r2score)


RMSE: 1.7114743355200899
R²: 0.971077700265979


In [122]:
result_df = pd.DataFrame({'Actual Sales': y_test, 'Predicted Sales': y_pred})
result_df.head()

,Actual Sales,Predicted Sales
150,22.510,22.515708
66,20.095,21.507639
114,26.399,25.684291
2,29.185,31.967049
15,36.229,37.489590


In [123]:
fig = px.scatter(result_df, x='Predicted Sales', y='Actual Sales', trendline='ols')
fig.update_layout(
    title='Actual vs Predicted Sales',
    xaxis_title='Predicted Sales',
    yaxis_title='Actual Sales'
)
fig.show()

Dapat kita lihat dari visualisasi perbandingan nilai prediksi dan nilai aslinya, setiap point menunjukan bahwa hasil prediksi hanya berbeda sedikit dari nilai aslinya.

# Melakukan prediksi harga terhadap mobil dengan spesifikasi yang telah ditentukan

In [130]:
data = {
    'Engine_size': [3.0],
    'Horsepower': [186.0],
    'Wheelbase' : [107.0],
    'Width' : [71.0],
    'Length' : [187.0],
    'Curb_weight' : [3.0],
    'Fuel_capacity' : [18.0],
    'Fuel_efficiency' : [24.0],
    'Power_perf_factor' : [77.0]
}

df_predict_price = pd.DataFrame(data)
df_predict_price

,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Power_perf_factor
0,3.0,186.0,107.0,71.0,187.0,3.0,18.0,24.0,77.0


In [131]:
price_pred = model.predict(df_predict_price)
print('Harga Mobil : {:.2f}'.format(price_pred[0]))


Harga Mobil : 26.40


Berdasarkan model regresi linear yang dibuat, spesifikasi mobil yang diberikan pada input menghasilkan prediksi harga sekitar USD 26.400. Nilai ini menunjukkan bahwa spesifikasi seperti berat mobil (Curb Weight), efisiensi bahan bakar (Fuel Efficiency), dan Power Performance Factor berkontribusi terhadap pembentukan harga. Artinya, mobil dengan kombinasi spesifikasi tersebut cenderung masuk ke segmen mid-range market, tidak terlalu murah namun juga belum masuk kategori premium.

Jadi, model dapat membantu memperkirakan harga mobil baru dengan cukup baik, meskipun faktor lain di luar spesifikasi teknis (seperti merek, tahun produksi, dan fitur tambahan) juga dapat memengaruhi harga aktual di pasar.